In [15]:
import re
import glob
import h5py
import numpy as np
import pandas as pd
import astropy.io.fits as pf
import matplotlib.pyplot as plt

rootdir = '/Users/peter/Projects/master-thesis/mag_errors/'
starlink_data = '/Users/peter/Projects/starlink_data/'

In [2]:
# Flux files
ff = np.sort(glob.glob(f'{rootdir}satflux/*.p'))

In [3]:
ff

array(['/Users/peter/Projects/master-thesis/mag_errors/satflux/satflux_LSC.p',
       '/Users/peter/Projects/master-thesis/mag_errors/satflux/satflux_LSE.p',
       '/Users/peter/Projects/master-thesis/mag_errors/satflux/satflux_LSN.p',
       '/Users/peter/Projects/master-thesis/mag_errors/satflux/satflux_LSS.p',
       '/Users/peter/Projects/master-thesis/mag_errors/satflux/satflux_LSW.p'],
      dtype='<U68')

In [4]:
for i in ff:
    x = pd.read_pickle(i)
    print(len(x))

0
168
291
437
450


In [5]:
fluxes = []
popts = []
pcovs = []
perrs = []
for i in ff[1:]:
    x = pd.read_pickle(i)
    lstseqs = x.keys()
    for lstseq in lstseqs:
        satnums = list(x[lstseq])[2:]
        for satnum in satnums:
            fluxes.append(x[lstseq][satnum]['satflux'])
            popts.append(x[lstseq][satnum]['popt'])
            pcovs.append(x[lstseq][satnum]['pcov'])
            perrs.append(x[lstseq][satnum]['perr'])
        

In [6]:
test = np.copy(fluxes)
for i, x in enumerate(test):
    test[i] = 10**x
    

In [7]:
print(np.std(fluxes))
print(np.std(test))

0.32922015919226466
63198.495750210684


In [8]:
def error_prop(logFlux, sigma_B):
    """
    m = -2.5log10(F) + B
    
    sigma_m = sqrt([(dm/dF)*sigma_F]**2 + [(dm/dB)*sigma_B]**2)
    
        => dm/dF = -2.5 * d/dF[log10(F)] = -2.5/F*ln(10)
        => dm/dB = 1
    
    sigma_m = sqrt([(-2.5/F*ln(10)) * sigma_F]**2 + [sigma_B]**2)
    
    sigma_F = uncertainty due to photon noise = np.std(fluxes)
    sigma_B = uncertainty due to background noise = pcov
    
    """
    sigma_F = 63198.495750210684
    F = 10**logFlux
    return np.sqrt(((-2.5/(F*np.log(10))) * sigma_F)**2 + (sigma_B)**2)
    

In [9]:
sigma_m = error_prop(fluxes[0], pcovs[0])
print(sigma_m)

1.6297948157081845


In [10]:
def error_prop_logged(F, sigma_B):
    """
    m = -2.5*F + B
    
    sigma_m = sqrt([(dm/dF)*sigma_F]**2 + [(dm/dB)*sigma_B]**2)
    
        => dm/dF = -2.5 
        => dm/dB = 1
    
    sigma_m = sqrt([(-2.5) * sigma_F]**2 + [sigma_B]**2)
    
    sigma_F = uncertainty due to photon noise = np.std(fluxes)
    sigma_B = uncertainty due to background noise = pcov
    
    """
    sigma_F = 0.32922015919226466
    sigma_m = np.sqrt(((-2.5) * sigma_F)**2 + (sigma_B)**2)
    print(sigma_m)

# Unfortunately, this isn't the correct way

In [11]:
error_prop_logged(fluxes[0], pcovs[0])

0.823051784963984


In [12]:
def error_prop_correct(logFlux, sigma_B):
    """
    m = -2.5*log10(F) + B
    
    sigma_m = sqrt([(dm/dF)*sigma_F]**2 + [(dm/dB)*sigma_B]**2)
    
        => dm/dF = -2.5/F*ln(10)
        => dm/dB = 1
        => sigma_F = 63198.495750210684
        => sigma_B = np.sqrt(pcov) = perr   

    sigma_m = sqrt([(-2.5/F*ln(10)) * sigma_F]**2 + [sigma_B]**2)
    sigma_F = uncertainty from flux => std of flux distribution provides an estimate
    sigma_B = uncertainty from background => std of the fit parameter B = perr 
    
    """
    sigma_F = 63198.495750210684
    F = 10**logFlux
    sigma_m = np.sqrt(((-2.5/(F*np.log(10))) * sigma_F)**2 + (sigma_B)**2)
    print(sigma_m)

# x = np.sqrt(pcovs[0])
error_prop_correct(fluxes[0], perrs[0])

1.630257604377099


## Including photon noise as uncertainty on flux

Error propagation:

m = -2.5 * x + B

=> x = log10(F) = flux

=> B = constant due to background

sigma_m = uncertainty on m, will depend on F and B according to:

=> sigma_m = sqrt( [d_m/d_F * sigma_F]^2 + [d_m/d_B * sigma_B]^2 )

dm/dF: -2.5 * d/dF[log10(F)] = -2.5/F*ln(10)

dm/dB: 1

sigma_B: perr = sqrt(pcov) = std of the fit parameter B

sigma_F: could define as std of F by collecting many measurements of F. But, doesn't take all sources into account. e.g. photon noise. Could assume sigma_F is completely due to photon noise. Could compute this by getting std of pixels surronding the satellite trail

In [13]:
import astropy.stats as aps
lstseq = 48506274
data = pf.getdata(f'{starlink_data}test_data/diff_images/LSC/diff_{lstseq}LSC.fits.gz')

In [14]:
x_min = 1136 
x_max = 869 
y_min = 925
y_max = 1293

if x_min < x_max:
    x0 = x_min
    x1 = x_max
else:
    x0 = x_max
    x1 = x_min
if y_min < y_max:
    y0 = y_min
    y1 = y_max
else:
    y0 = y_max
    y1 = y_min
    
def fitmagfunc(x, b):
    # magnitude = -2.5 * np.log10(flux) + constant (from background noise)
    return -2.5 * x + b    

def brightness_err(logF, sigma_F, sigma_B):
    F = 10**logF
    dm_dF = -2.5/(F*np.log(10))
    dm_dB = 1
    sigma_m = np.sqrt( (dm_dF * sigma_F)**2 + (dm_dB * sigma_B)**2 )
    return sigma_m

In [15]:
# Brightness magnitude
m = fitmagfunc(fluxes[0], popts[0])

# Uncertainty due to photon noise = sigma_F
mean, median, sigma_F = aps.sigma_clipped_stats(np.fabs(data[y0:y1, x0:x1]))

# Uncertainty due to background noise
sigma_B = np.sqrt(pcovs[0])

# Now propagate error to estimate uncertainty in the brightness magnitude
sigma_m = brightness_err(fluxes[0], sigma_F, sigma_B)

print(f'm = {round(m,4)} +/- {round(sigma_m,4)}')

m = 5.4297 +/- 0.0389


In [26]:
p = pd.read_pickle(f'{starlink_data}full_data/passages_20221023LSC.p')
p['48506345']

{'47772U': {'start': {'jd': 2459876.522254584,
   'lst': 21.978274854520222,
   'ra': 311.63254697650916,
   'dec': -49.43236302592727,
   'x': 2643.497293869421,
   'y': 2219.0567380831817},
  'end': {'jd': 2459876.5224023364,
   'lst': 21.981820919742177,
   'ra': 321.3814348461688,
   'dec': -43.92129607847961,
   'x': 2327.369803741676,
   'y': 1905.003180617398},
  'JD': 2459876.522291522},
 '49450U': {'start': {'jd': 2459876.522254584,
   'lst': 21.978274854520222,
   'ra': 303.995480721863,
   'dec': -32.56216180525492,
   'x': 3069.235754103156,
   'y': 1427.4979091994649},
  'end': {'jd': 2459876.5224023364,
   'lst': 21.981820919742177,
   'ra': 311.9831545763556,
   'dec': -26.650709545357753,
   'x': 2754.688453914375,
   'y': 1089.8309555697506},
  'JD': 2459876.522291522}}

In [16]:
def get_tles():
    
    # Load TLEs for all satellite passages
    satfiles = f"{starlink_data}/test_data/3leComplete.txt"
    with open(satfiles) as f:
        all_tles = f.readlines()
        f.close()

    # Split TLE list into individual lists for each TLE
    all_tles = [i.strip() for i in all_tles]
    tles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

    # Reduce TLEs to Starlink only
    starlink_tles = []
    for tle in tles:
        if "STARLINK" in tle[0]:
            starlink_tles.append(tle)
            
    for tle in starlink_tles:
        tle[0] = tle[0][2:]
        
    return starlink_tles

tles = get_tles()

In [31]:
def starlink_df(tles, save=False):
    
    names = []
    satnums = []
    for tle in tles:
        names.append(tle[0])
        satnums.append(tle[1][2:8])
        
    df = pd.DataFrame({'name': names, 'num': satnums})
    
    if save:
        df.to_pickle('starlink_names.p')
    return df

df = starlink_df(tles)

In [38]:
sat = '44238U'
name = df.loc[df['num'] == sat, 'name'].values[0]
print(name)

STARLINK-24


In [26]:
df

name     num
0       STARLINK-24  44238U
1       STARLINK-71  44252U
2     STARLINK-1007  44713U
3     STARLINK-1008  44714U
4     STARLINK-1009  44715U
...             ...     ...
3174  STARLINK-5124  54011U
3175  STARLINK-5126  54012U
3176  STARLINK-5098  54013U
3177  STARLINK-5116  54014U
3178  STARLINK-5055  54015U

[3179 rows x 2 columns]

### Could include uncertainties for all the stellar fluxes

In [138]:
lc = h5py.File(f'{starlink_data}test_data/fast_20221023LSC.hdf5')
lc.keys()

<KeysViewHDF5 ['astrometry', 'header', 'lightcurves', 'stars', 'station']>

In [176]:
lc['lightcurves']['1188881'].dtype.fields.keys()

dict_keys(['lstseq', 'sky', 'esky', 'peak', 'x', 'y', 'pflag', 'aflag', 'tmpmag0', 'tmpemag0', 'cflag', 'flux0', 'flux1', 'eflux0', 'eflux1'])

In [177]:
lc['lightcurves']['1188881']['eflux1']

array([219.93915, 221.60266, 219.29958, 223.87366, 222.74988, 225.32402,
       221.58476, 212.9958 , 226.11504, 220.81712, 223.28064, 221.30785,
       225.69043, 224.79608, 225.7578 , 227.0835 , 227.69464, 221.47737,
       229.92023, 226.71748, 224.8624 , 227.03719, 227.72093, 226.2408 ,
       221.90656, 230.79414, 227.77919, 223.47311, 223.8087 , 231.99716,
       228.83308, 233.85834, 225.295  , 221.20934, 219.96484, 226.56836],
      dtype=float32)

In [178]:
lc.close()

In [183]:
import bringreduce.configuration as cfg
cfg.initialize('20221023LSC')
starcat = pf.getdata(cfg.starcat)

In [192]:
np.sort(starcat.dtype.names)

array(['ASCC', 'Blend', 'Bmag', 'Dist10', 'Dist6', 'Dist8', 'Duplicate',
       'GDE', 'GRA', 'Gmag', 'HD', 'HIP', 'Inclusion', 'Par', 'SKYIDX',
       'SpType', 'TYC1', 'TYC2', 'VSXid', 'Var', 'Vmag', '_DEJ2000',
       '_RAJ2000', 'ePar'], dtype='<U9')

In [191]:
starcat['Var']

array([20,  6, 16, ...,  0,  0,  4], dtype=uint8)

## Sources of uncertainty from difference image (repeated 10 times)

In [33]:
files = glob.glob(f'{starlink_data}error_tests/*.fits.gz')
files = sorted(files, key=lambda x: int(re.search(r'\d+', x).group()))

In [35]:
%matplotlib qt
fig, axs = plt.subplots(3, 3, figsize=(12, 12))

for i, ax in enumerate(axs.flat):
    img = pf.getdata(files[i])
    ax.imshow(img, vmin=-15, vmax=100, cmap='terrain')
    ax.axis('off')

plt.tight_layout()
plt.savefig('error_tests_5.png', facecolor='w', bbox_inches='tight', dpi=300)

In [30]:
def calculate_statistics(pixels):
    mean = np.mean(pixels)
    sdev = np.std(pixels)
    median = np.median(pixels)
    return mean, sdev, median

def evaluate_noise(img):
    pixels = img.flatten()
    noise_sdev = np.std(pixels)
    return noise_sdev

def noise_sources(images, threshold):
    # Need to define the expected noise characteristics i.e. a threshold
    for img in images:
        noise_sdev = evaluate_noise(img)
        if noise_sdev > threshold:
            print("Readout noise")
        else:
            print("Other")

In [36]:
for i, f in enumerate(files, start=1):
    print(f'Image {i}')
    img = pf.getdata(f)
    pixels = img.flatten()
    mean, sdev, median = calculate_statistics(pixels)
    print("Mean:", mean)
    print("Standard Deviation:", sdev)
    print("Median:", median)

Image 1
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 2
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 3
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 4
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 5
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 6
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 7
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 8
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 9
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0
Image 10
Mean: -0.011657438282518468
Standard Deviation: 33.479822110727675
Median: 0.0


Since all 10 delta images have the exact same mean, sdev, and median for the pixel values, this suggests that the noise characteristics in those images are consistent. 

i.e. this implies that the noise sources contributing to the delta images, such as photon noise, readout noise, dark current noise, and systematic noise, are relatively stable and have similar magnitudes across the dataset.

Photon Noise: If the standard deviation of the pixel values is consistent across the delta images, it suggests that the photon noise contribution to the images is stable. Photon noise is generally proportional to the square root of the signal level, and a consistent standard deviation indicates a consistent signal level across the images.

Readout Noise: The identical mean, standard deviation, and median values across the delta images suggest that the readout noise component is relatively constant. Readout noise arises from the electronics in the image acquisition process and is typically independent of the signal level. Therefore, the consistent noise characteristics indicate a stable readout noise contribution.

Dark Current and Systematic Noise: If the noise characteristics are identical across the delta images, it suggests that the contributions from dark current noise and systematic noise are also consistent. Dark current noise is caused by thermal effects in the camera sensor and can vary with exposure time and temperature. However, if the noise characteristics are consistent, it implies a stable dark current contribution. Systematic noise, arising from calibration errors or instrumental effects, would also be consistent across the images.

In summary, the consistent noise characteristics in the delta images indicate stability in the noise sources and suggest that the observed variations are primarily due to the satellite trails rather than noise fluctuations. This provides a more reliable foundation for analyzing and quantifying the impact of satellite trails in observations.